In [3]:
import numpy as np
np.set_printoptions(suppress=True) #Evita notación científica
import math

# FUNCIONES
# !pip install import_ipynb
# !pip show import_ipynb
import import_ipynb
import G360_Function as func # Assuming G360_Function.ipynb is in the same directory

import G360_Parameter as param
from collections.abc import Iterable

In [13]:
# FORMATO PARA MANEJO DE VECTORES
class FReaction(np.ndarray):
    def __new__(cls, f_v, f_x=None, f_y=None):
        if (f_x != None) & (f_y != None):
            obj = np.array([f_v, f_x, f_y], dtype=float).view(cls)
        else:
            obj = np.array(f_v, dtype=float).view(cls)
        return obj
        
    def __str__(self):
        return repr(self)

    @property
    def F_vert(self):
        return self[0]

    @property
    def F_x_long(self):
        return self[1]

    @property
    def F_y_transv(self):
        return self[2]
#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#
# eg REACCIONES
#DIAGONAL DIR PATA
FC_diag = [-800, +12, +23]
FT_diag = [+650, -15, +10]
Fl_diag = [+000, +30, -20]
Ft_diag = [+200, -10, -35]
FR_diag = FReaction([FC_diag, FT_diag, Fl_diag, Ft_diag])

#ORTOGONAL
FC_ort = [-730, +25, +50]
FT_ort = [+600, -30, +20]
Fl_ort = [+000, +60, -40]
Ft_ort = [+150, -20, -70]
FR_ort = FReaction([FC_ort, FT_ort, Fl_ort, Ft_ort])

FR_ortM, FR_diagM = 1.5*FR_ort, 1.5*FR_diag

#eg
input_g = {
    'ancho_stub' : 0.31 ,
    'long_stub' : 2.0 ,
    'sep_patas' : 4.0 ,
    'ang_bidireccional' :  math.radians(5),
    'ang_tridimensional' : None
}
input_g['ang_tridimensional'] = func.ang_bid_to_trid(input_g['ang_bidireccional'])

input_z = {
    'lista_ped': [0.25, 0.5, 0.75, 1.0], 
    'recub_dis': 0.05, 
    'recub_ped': 0.05,
    'pata_comp':  1,
    'pata_trac':  4,
    'Esp_solado': 0.05,
    'Geom_Ped' : 0  , #  CUADRADO == 0 , CIRCULAR == 1
    'cuantia_min_losa' : 0.0018 ,
    'cuantia_min_ped' : 0.005   #0.5%
}
PEDESTAL_max = max(input_z['lista_ped'])
PEDESTAL_min = min(input_z['lista_ped'])

## ZAPATA AISLADA

In [8]:
# eg. variables
input_ = {
    'Cohesion': 10.0,
    'Gamma_suelo': 16.2,
    'Phi': math.radians(30),
    'Sigma_adm': 100.0}
inputOP_ = {
    'Betha': 2/3 * input_['Phi'],
    'Comportamiento_suelo': 0, # GRANULAR == 0 , COHESIVO == 1
    'Gamma_concr': 24,
    'Gamma_suelo_rell': 16.2,
    'Prof_desprecio': 0.0,
    'Suelo_sumergido': 0, # SECO == 0 , SUMERGIDO == 1
    'fprimac': 21.0,
    'fy': 420.0,
    'kp': 3.0}

#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#
# OBJETO PARA CIMENTACIÓN TIPO ZAPATA AISLADA
class CimentacionZapata():
    def __init__(self, AP_AlturaPedestal, P_Profundidad, A_Ancho, EP_EspesorPedestal, E_EspesorZap):
        self._AP_AlturaPedestal = AP_AlturaPedestal
        self._P_Profundidad = P_Profundidad
        self._A_Ancho = A_Ancho
        self._EP_EspesorPedestal = EP_EspesorPedestal
        self._E_EspesorZap = E_EspesorZap
####
# PROPIEDADES
    @property
    def g_prof_pedestal(self):
        return self._P_Profundidad - self._E_EspesorZap
    @property
    def g_prof_zapata(self):
        return self._P_Profundidad

    @property
    def g_ancho_pedestal(self):
        return self._EP_EspesorPedestal
    @property
    def g_ancho_zapata(self):
        return self._A_Ancho

    @property
    def g_esp_zapata(self):
        return self._E_EspesorZap  
    @property
    def g_alt_pedestal(self):
        return self._AP_AlturaPedestal
    #Hide
    @property
    def _Pos_Ped(self):
        return np.array([self._AP_AlturaPedestal + self._P_Profundidad, 
                         self._A_Ancho/2, 
                         self._A_Ancho/2])
        
####
# GEOMETRÍA Y CANTIDADES
    def volumen_concr(self):
        self._secc_ped = self._EP_EspesorPedestal**2 if input_z['Geom_Ped'] == 0 else math.pi * (self._EP_EspesorPedestal/2)**2
        self._volumen_concr = self._A_Ancho**2 * self._E_EspesorZap + self._secc_ped * (self._AP_AlturaPedestal + self._P_Profundidad - self._E_EspesorZap)
        return self._volumen_concr

    def volumen_excavacion(self):
        self._volumen_excavacion = self._A_Ancho**2 * self._P_Profundidad
        return self._volumen_excavacion

    def volumen_relleno(self):
        self._volumen_relleno = self.volumen_excavacion() - (self.volumen_concr() - self._secc_ped * self._AP_AlturaPedestal)
        return self._volumen_relleno

    def volumen_cono(self):
        self._R0_cono = self._P_Profundidad * math.tan(inputOP_['Betha']) 
        self._volumen_cono = func.troncopiramidal(self._A_Ancho**2, 
                                                  (self._A_Ancho + 2 * self._R0_cono)**2, 
                                                  self._P_Profundidad) - ( self._A_Ancho**2 * self._P_Profundidad)
        return self._volumen_cono
        
#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#
# CREACIÓN DE OBJETO
_A = 2.4
_P = 3
_AP  = 1
_EP = .5
_E = .4

z1 = CimentacionZapata(_AP, _P, _A, _EP, _E)

### CHEQUEO

In [11]:
# OBJETO PARA LOS CHEQUEOS EN LAS ZAPATAS
class ChequeoZapataACI(): # SE INGRESA EL OBJETO Y OBJETOS FReaction
    def __init__(self, Obj_Zap, reaccion_ortogonal, reaccion_diagonal, reaccion_ortogonal_MAYORADA, reaccion_diagonal_MAYORADA):
        self._obj_Zap = Obj_Zap #obj
        self._ort =  func.abs_vectores(reaccion_ortogonal) #F([])
        self._diag = func.abs_vectores(reaccion_diagonal)  #F([])

        self._ortM =  func.abs_vectores(reaccion_ortogonal_MAYORADA) #F([])
        self._diagM = func.abs_vectores(reaccion_diagonal_MAYORADA)  #F([])

        #REACCIONES VERTICALES
        self._peso_concreto = - np.array([self._obj_Zap.volumen_concr(), 0, 0]) * inputOP_['Gamma_concr'] #[]
        self._peso_suelo = -(np.array([self._obj_Zap.volumen_relleno(), 0, 0]) +  
                            self._obj_Zap.volumen_cono() * func.casos_traccion(self._ort) ) * inputOP_['Gamma_suelo_rell'] # ([])

        self._ort_RT = (self._ort +   # REACCIONES
                        self._peso_concreto  + # CONCRETO
                        self._peso_suelo ) # SUELO  # F([])
        
        # REACCIONES HORIZONTALES
        self._empuje_pasivo_ped  = func.empuje_pasivo(q1 = input_['Gamma_suelo'] * self._obj_Zap.g_prof_pedestal, 
                                                      q2 = 0, 
                                                      Area = self._obj_Zap.g_prof_pedestal * self._obj_Zap.g_ancho_pedestal, 
                                                      kp = inputOP_['kp']) /param.FS_EMPUJE_PERM  # EMPUJE PEDESTAL
        self._empuje_pasivo_ped  = np.array([0, self._empuje_pasivo_ped, self._empuje_pasivo_ped]) #[]
        self._empuje_pasivo_losa = func.empuje_pasivo(q1 = input_['Gamma_suelo'] * self._obj_Zap.g_prof_pedestal, 
                                                      q2 = input_['Gamma_suelo'] * self._obj_Zap.g_prof_zapata,
                                                      Area = self._obj_Zap.g_esp_zapata * self._obj_Zap.g_ancho_zapata, 
                                                      kp = inputOP_['kp']) /param.FS_EMPUJE_PERM  # EMPUJE LOSA INF
        self._empuje_pasivo_losa = np.array([0, self._empuje_pasivo_losa, self._empuje_pasivo_losa]) #[]
        
        self._friccion_base = -self._ort_RT[:,0] * math.tan(input_['Phi'])/param.FS_FRICCION_PERM
        self._friccion_base = np.array([[0,0,0,0], self._friccion_base, self._friccion_base  ]).T #([])
        self._cohesion_base = self._obj_Zap.g_ancho_zapata **2 * input_['Cohesion'] / param.FS_COHESION_PERM
        self._cohesion_base = np.array([0, self._cohesion_base, self._cohesion_base]) #[]
        self._reaccion_base = self._friccion_base if  inputOP_['Comportamiento_suelo'] == 0 else self._cohesion_base 

        # ACTUALIZAR VECTOR RESULTANTE
        self._ort_RT = (self._ort_RT   * np.array([1,-1,-1])+   # REACCIONES
                        self._empuje_pasivo_ped + self._empuje_pasivo_losa  + # EMPUJE PASIVO DEL SUELO
                        self._reaccion_base ) # APORTE POR FRICCION Y COHESIÓN   # F([])

        # MOMENTOS GENERADOS
        # COMPONENTES KP
        _Vector_kp_ped  = self._empuje_pasivo_ped[1]
        _Pos_kp_ped = self._obj_Zap.g_prof_pedestal/3 + self._obj_Zap.g_esp_zapata
        _Vector_kp_losa = self._empuje_pasivo_losa[1]
        _Pos_kp_losa = func.cg_trapecio(input_['Gamma_suelo'] * self._obj_Zap.g_prof_zapata,
                                       input_['Gamma_suelo'] * self._obj_Zap.g_prof_pedestal,
                                       self._obj_Zap.g_esp_zapata)
        
        # COMPONENTES REACCIONES VERTICALES - SERVICIO
        _f_comp =(self._ort * func.casos_compresion(self._ort) +   # REACCIONES
                  self._peso_concreto  + # CONCRETO
                  self._peso_suelo ) # SUELO  # F([])
        _f_comp = np.array([[i.F_vert, i.F_vert]  for i in _f_comp])
        _f_comp = abs(_f_comp)
        _f_tr = self._ort * func.casos_traccion(self._ort)
        _f_tr = np.array([[i.F_vert, i.F_vert]  for i in _f_tr])

        # COMPONENTES Fx,y
        _Fx_y = self._ort[:, (1,2)]
        _h_PR = (self._obj_Zap.g_prof_zapata + self._obj_Zap.g_alt_pedestal)
        
        # COMPUTO
        self._Mvolc = (_f_tr * self._obj_Zap.g_ancho_zapata/2 +
                  _Fx_y * _h_PR)
        self._Mestab = (_f_comp * self._obj_Zap.g_ancho_zapata/2 +
                   _Vector_kp_ped * _Pos_kp_ped +
                   _Vector_kp_losa * _Pos_kp_losa )


        # COMPONENTES REACCIONES VERTICALES - MAYORADAS
        _f_compM = self._ortM_RT =(self._ortM * func.casos_compresion(self._ortM) +   # REACCIONES
                                   self._peso_concreto  + # CONCRETO
                                   self._peso_suelo ) # SUELO  # F([])
        _f_compM = np.array([[i.F_vert, i.F_vert]  for i in _f_compM])
        _f_compM = abs(_f_compM)
        _f_trM = self._ortM * func.casos_traccion(self._ortM)
        _f_trM = np.array([[i.F_vert, i.F_vert]  for i in _f_trM])

        # COMPONENTES Fx,y
        _Fx_yM = self._ortM[:, (1,2)]
        _h_PR = (self._obj_Zap.g_prof_zapata + self._obj_Zap.g_alt_pedestal)
        
        # COMPUTO
        self._MvolcM  = (_f_trM * self._obj_Zap.g_ancho_zapata/2 +
                         _Fx_yM * _h_PR)
        self._MestabM = (_f_compM * self._obj_Zap.g_ancho_zapata/2 +
                         _Vector_kp_ped * _Pos_kp_ped +
                         _Vector_kp_losa * _Pos_kp_losa )

        # Distancia d de diseño
        self._d_losa = self._obj_Zap.g_esp_zapata  - input_z['recub_dis']
        self._d_ped  = self._obj_Zap.g_ancho_pedestal - input_z['recub_ped']
####
# CHEQUEOS DEL SUELO
    def FS_arranc(self):
        _fuerzas_traccion = self._ort * func.casos_traccion(self._ort) #F([])
        return ["comp" if self._ort[i].F_vert <= 0 else abs(self._ort_RT[i].F_vert - _fuerzas_traccion[i].F_vert)/_fuerzas_traccion[i].F_vert for i in range(4)]
    def FS_arranc_Boole(self):
        return func.min_(self.FS_arranc()) >= param.FS_ARRANC
#
    def FS_vuelco(self):
        _FS = self._Mestab/self._Mvolc
        return [min(i) for i in _FS]
    def FS_vuelco_Boole(self):
        return func.min_(self.FS_vuelco()) >= param.FS_VOLCAM
#
    def FS_deslizamiento(self):
        _reacciones_fx_fy = self._ort_RT[:, (1,2)]
        _minima_reaccion_fx_fy = func.min_(_reacciones_fx_fy)
        if _minima_reaccion_fx_fy == 0:
            return [param.FS_DESLIZAM for i in range(4)]
        elif _minima_reaccion_fx_fy < 0:
            return [0 for i in range(4)]
        else:
            _FS_matriz = (self._ort_RT + self._ort)[:, (1,2)] / (self._ort)[:, (1,2)]
            return [min(i) for i in _FS_matriz]
    def FS_deslizamiento_Boole(self):
        return func.min_(self.FS_deslizamiento()) >= param.FS_DESLIZAM
#
    def FS_carga_admisible_suelo(self):
        _exc = self._obj_Zap.g_ancho_zapata/2 - ( self._Mestab - self._Mvolc  )/ abs(self._ort_RT[:,0])[:, np.newaxis]  # [:, np.newaxis] COMANDO PARA VALIDEZ DIMENSIONAL
        _sigma = func.sigma_comp(Fv = abs(self._ort_RT[:,0]), 
                                 Ancho = self._obj_Zap.g_ancho_zapata, 
                                 exc = _exc) 
        _esfuerzo_max = [i[0] for i in _sigma]
        return _esfuerzo_max / input_['Sigma_adm']
    def FS_carga_admisible_suelo_Boole(self):
        return func.max_(self.FS_carga_admisible_suelo()) <= 1
        
#flagg_Jc PASAR LAS ECUACIONES DE NORMA CON LITERAL Y FORMA DE ESCRITURA PARA REFERENCIAS FUTURAS
####
# CHEQUEOS DE DISEÑO ESTRUCTURAL
    def _esfuerzo_sigma_disenho(self):
        _exc = self._obj_Zap.g_ancho_zapata/2 - ( self._MestabM - self._MvolcM  )/ abs(self._ortM_RT[:,0])[:, np.newaxis]  # [:, np.newaxis] COMANDO PARA VALIDEZ DIMENSIONAL
        _sigma = func.sigma_comp(Fv = abs(self._ortM_RT[:,0]), 
                                 Ancho = self._obj_Zap.g_ancho_zapata, 
                                 exc = _exc) 
        
        # esfuerzos sigma:       max,  prom_losa , prom_cara 
        _esfuerzos = [[_sigma[i][0] , 
                       (_sigma[i][0]+_sigma[i][3])/2 - input_['Gamma_suelo'] * self._obj_Zap.g_prof_zapata ,
                       (_sigma[i][0] + max(_sigma[i][1], _sigma[i][2]) )/2] for i in range(4)]
        return np.array(_esfuerzos)
#
    def FS_punzonamiento(self):
        # flagg_Jc VERIFICAR GEOMETRÍA CIRCULAR
        # 'Geom_Ped' : 0   #  CUADRADO == 0 , CIRCULAR == 1
        b0 =  (self._obj_Zap.g_ancho_pedestal + self._d_losa)*4   if input_z['Geom_Ped'] == 0 else (self._obj_Zap.g_ancho_pedestal + self._d_losa)/2 * math.pi
        A0 =  (self._obj_Zap.g_ancho_pedestal + self._d_losa)**2  if input_z['Geom_Ped'] == 0 else (self._obj_Zap.g_ancho_pedestal + self._d_losa)**2/4 * math.pi 
        
        Vu_actuante = self._esfuerzo_sigma_disenho()[:, 1] * (self._obj_Zap.g_ancho_zapata**2 - A0)
        sigma_act = Vu_actuante/(b0 * self._d_losa * 1000)
        sigma_adm = min(1/3 * math.sqrt(inputOP_['fprimac']) , 
                        1/6 * (3) * math.sqrt(inputOP_['fprimac']) ,
                        1/12 * (40 * self._d_losa * b0 +2) * math.sqrt(inputOP_['fprimac']) )* param.φ_cortante
        # Vc_resist = b0 * sigma_adm*1000 * self._d_losa
        return sigma_adm/sigma_act
    def FS_punzonamiento_Boole(self):
        return func.min_(self.FS_punzonamiento()) >= 1
#
    def FS_cortante(self):
        Vu_actuante = (self._esfuerzo_sigma_disenho()[:,2] * self._obj_Zap.g_ancho_zapata +
                      - inputOP_['Gamma_concr'] * self._obj_Zap.g_esp_zapata +
                      - inputOP_['Gamma_suelo_rell'] * self._obj_Zap.g_prof_pedestal+
                       0)*( self._obj_Zap.g_ancho_zapata/2 - self._obj_Zap.g_ancho_pedestal/2 - self._d_losa)
        sigma_act = Vu_actuante/(self._obj_Zap.g_ancho_zapata * self._d_losa * 1000)
        sigma_adm = param.φ_cortante * 1/6 * math.sqrt(inputOP_['fprimac'])
        return sigma_adm/sigma_act
    def FS_cortante_Boole(self):
        return func.min_(self.FS_cortante()) >= 1
#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#
# CREACIÓN DE OBJETO
che = ChequeoZapataACI(z1, FR_ort, FR_diag, FR_ortM, FR_diagM)

`Suelo aportante para casos a tracción:`


\begin{align}
{Traccion} & = Relleno + Cono \\
{Compresion} & = Relleno
\end{align}

`def FS_arranc(self):`

\begin{align}
R = (-F↑) + F↑ & \\
\dfrac{F↓}{F↑} & \ge 1.5 \\
\dfrac{(-F↑)}{F↑} & \le 1.5 \\
\dfrac{R - F↑}{F↑} & \le 1.5 
\end{align}

$(-F↑)$ = Fuerzas a compresión (Suelo y concreto)

$F↑$ = Reacción vertical a tracción

`def FS_vuelco`
\begin{align}
\dfrac{M_{estab}}{M_{volc}}   & \ge 1.5
\end{align}
$M_{estab} = F_{kp} * cg_{kp} + F_{comp + suelo + concr} * A/2$ Momentos estabilizadores

$M_{volc} = F_{x,y} * h_{ped} + F_{tracc} * A/2$ Momentos que generan volcamiento en la cimentación

`def FS_deslizamiento`
\begin{align}
R = (F_{kp} + F_{base}) + F_{x, y} & \\
\dfrac{F_{resistentes}}{F_{horizontales}} & \ge 1.5 \\
\dfrac{R - F_{x, y}}{F_{x, y}} & \ge 1.5
\end{align}



$F_{kp}$  Resistencia del empuje pasivo del suelo

$F_{base}$ Reacción en la base generada por la fricción o cohesión

`FS_carga_admisible_suelo`

\begin{align}
exc_{x,y} & =  \dfrac{A}{2} - \dfrac{M_{estab} - M_{volc}}{F_v}  \\
\sigma_{1, 2, 3, 4} &= \dfrac{F_v}{A^2} \cdot \left(  1 \pm 6 \cdot \dfrac{exc_x}{A} \pm 6 \cdot  \dfrac{exc_y}{A}       \right) \\
 \\
\sigma_{max} &= \sigma_1\\
\sigma_{prom-losa} &= \dfrac{\sigma_1 + \sigma_4}{2} - \gamma_{suelo} \cdot h_{desplante}\\
\sigma_{prom-cara} &= \dfrac{\sigma_1 + max(\sigma_2, \sigma_3)}{2}
\end{align}


`VERIFICACIONES DE DISEÑO SEGÚN ACI 318:`


### DISEÑO DE ACERO

In [497]:
# OBJETO PARA EL DISEÑO DE ACERO EN LA ZAPATA
class AceroZapataACI():
    def __init__(self, obj_Zap, cheq_Zap, reaccion_ortogonal_MAYORADA, reaccion_diagonal_MAYORADA):

        self._obj_Zap = obj_Zap #obj
        self._cheq_Zap = cheq_Zap #cequeos
        
        self._ortM =  func.abs_vectores(reaccion_ortogonal_MAYORADA) #F([])
        self._diagM = func.abs_vectores(reaccion_diagonal_MAYORADA)  #F([])

# DISEÑO DEL PEDESTAL
    def Dis_ped(self):
        # REACCIONES CONSIDERANDO ESBELTEZ
        _V = func.Mom_viga_esbelta(Geometria = input_z['Geom_Ped'] , 
                                   fprima_c = inputOP_['fprimac'] , 
                                   fy = inputOP_['fy'], 
                                   ancho = self._obj_Zap.g_ancho_pedestal , 
                                   rr = input_z['recub_ped'] , 
                                   luz = self._obj_Zap.g_prof_pedestal + self._obj_Zap.g_alt_pedestal , 
                                   Vector = self._diagM.copy() )
        # CALCULO DE REFUEZO A FLEXO TRACCIÓN
        _ref1 = func.Ref_flexotrac(Ps = abs(_V[:,0]) , 
                                   Mc2 = _V[:,2] , 
                                   D =  self._obj_Zap.g_ancho_pedestal,  
                                   recub = input_z['recub_ped'] , 
                                   fy = inputOP_['fy'], 
                                   fc = inputOP_['fprimac']  )
        _ref1 *= func.casos_compresion(self._diagM)[:,0]
        _ref2 = input_z['cuantia_min_ped']* (self._obj_Zap.g_ancho_pedestal*100)**2
        Acero_min_flexotrac = max(max(_ref1) , _ref2)
        
        
        return 

Ace = AceroZapataACI(z1, che, FR_ortM, FR_diagM)

In [499]:
Ace.Dis_ped()

# (_V[:,0] , 
#  _V[:,2] , 
# z1.g_ancho_pedestal,  
#  input_z['recub_ped'] , 
#  inputOP_['fy'], 
#  inputOP_['fprimac'])

68.55497511377772

## ZAPATA CUÑA

In [227]:
##########
# IMPORTADO PARA SUELOS EN LA CLASIFICIACIÓN DENTRANSELEC
tipo_de_suelo = '4C'
dic_suelo = param.SUELOS_CHILE[str(tipo_de_suelo)]

#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#
# eg. variables
input_ = {
    'Cohesion':             dic_suelo['C'],
    'Gamma_suelo_aplast':   dic_suelo['SueloAplast'],
    'Gamma_suelo_arranc':   dic_suelo['SueloArranc'],
    'Phi':                  math.radians(dic_suelo['phi']),
    'Sigma_adm':            dic_suelo['CapAdm']}
inputOP_ = {
    'Betha':                math.radians(dic_suelo['betha']),
    'Comportamiento_suelo': 1,  # GRANULAR == 0 , COHESIVO == 1
    'Gamma_concr_aplast':   dic_suelo['ConcrAplast'],
    'Gamma_concr_arranc':   dic_suelo['ConcrArranc'],
    'Gamma_suelo_' :        None, # Variables opcionales para diseño específico
    'Gamma_suelo_rell':     None, # Variables opcionales para diseño específico
    'Suelo_sumergido':      1,  # SECO == 0 , SUMERGIDO == 1
    'fprimac':              21.0,
    'fy':                   420.0,
    'kp':                   round(math.tan( math.pi/4 + input_['Phi']/2 )**2, 2)}

#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#
# OBJETO PARA CIMENTACIÓN ZAPATA TIPO CUÑA
class CimentacionCunha():
    def __init__(self, AP_AlturaPedestal, P_Profundidad, A_Ancho, EP_EspesorPedestal, E_EspesorZap, Hy_DesnivelIncl, 
                 Hx_DesnivelIncl = None, Alpha_AngIncl = None, F_ProfundidadCunha = None, Cunha = None):
        self._AP_AlturaPedestal = AP_AlturaPedestal
        self._P_Profundidad = P_Profundidad
        self._A_Ancho = A_Ancho
        self._EP_EspesorPedestal = EP_EspesorPedestal
        self._E_EspesorZap = E_EspesorZap
        self._Hy_DesnivelIncl = Hy_DesnivelIncl
        
        # DEFINICIÓN DE VARIABLES OPCIONALES
        self._Alpha_AngIncl = math.radians(60) if Alpha_AngIncl == None else math.radians(Alpha_AngIncl) #rads
        if Hx_DesnivelIncl == None:
            self._Hx_DesnivelIncl = func.mround_down( self._Hy_DesnivelIncl/math.tan(self._Alpha_AngIncl) , 5/100) #Redondear hacia abajo en multiplos de 5 cm
        else:
            self._Hx_DesnivelIncl = Hx_DesnivelIncl
            self._Alpha_AngIncl = math.atan(self._Hy_DesnivelIncl / self._Hx_DesnivelIncl)

        self._F_ProfundidadCunha = 0.3 if F_ProfundidadCunha == None else F_ProfundidadCunha

        # VARIABLES CALCULADAS
        self._EPinf_EspesorPedestal_inf = self._A_Ancho  - 2*self._Hx_DesnivelIncl
        self._Cunha_OP = ( self._EPinf_EspesorPedestal_inf - self._EP_EspesorPedestal) - func.proporcion(self._EPinf_EspesorPedestal_inf - self._EP_EspesorPedestal,
                                                                                                      PEDESTAL_max + self._F_ProfundidadCunha + ( self._P_Profundidad - self._F_ProfundidadCunha)/3,
                                                                                                      PEDESTAL_max + self._F_ProfundidadCunha)
        self._Cunha_OP = max(func.mround_down(self._Cunha_OP, 0.05), 10/100)
        self._Cunha = self._Cunha_OP if (Cunha == None) else Cunha
        self._Cunha_nvl_terreno = func.proporcion(self._EPinf_EspesorPedestal_inf - self._EP_EspesorPedestal - self._Cunha,
                                                 self._AP_AlturaPedestal + self._F_ProfundidadCunha,
                                                 self._AP_AlturaPedestal)
####
# PROPIEDADES
    @property
    def g_prof_pedestal(self):
        return self._P_Profundidad - self._E_EspesorZap
    @property
    def g_prof_zapata(self):
        return self._P_Profundidad

    @property
    def g_altura_pedestal(self):
        return self._AP_AlturaPedestal
        
    @property
    def g_ancho_pedestal(self):
        return self._EP_EspesorPedestal
    @property
    def g_ancho_zapata(self):
        return self._A_Ancho

    @property
    def g_esp_zapata(self):
        return self._E_EspesorZap    

    @property
    def g_excentricidad(self):
        return round(param.PR*(math.tan(input_g['ang_tridimensional']))/(2**0.5),3)+((self._EPinf_EspesorPedestal_inf-self._EP_EspesorPedestal)/2)

####
# CANTIDADES
    def volumen_concr(self):
        self._v_losa = self._A_Ancho**2 * self._E_EspesorZap
        self._v_Incl = func.troncopiramidal(self._A_Ancho**2, self._EPinf_EspesorPedestal_inf**2, self._Hy_DesnivelIncl)
        self._v_cunha = self._EPinf_EspesorPedestal_inf**2 * (self._P_Profundidad - self._F_ProfundidadCunha - self._Hy_DesnivelIncl - self._E_EspesorZap)
        self._v_ped_cunha = func.troncopiramidal(self._EP_EspesorPedestal**2 , (self._EPinf_EspesorPedestal_inf - self._Cunha)**2, self._AP_AlturaPedestal + self._F_ProfundidadCunha)
        return self._v_losa + self._v_Incl + self._v_cunha + self._v_ped_cunha

    def volumen_excavacion(self):
        if self._Alpha_AngIncl >= math.radians(60):
            self._v_losa = self._A_Ancho**2 * self._E_EspesorZap
            self._v_Incl = func.troncopiramidal(self._A_Ancho**2, self._EPinf_EspesorPedestal_inf**2, self._Hy_DesnivelIncl)
            self._v_cunha = self._EPinf_EspesorPedestal_inf**2 * (self._P_Profundidad  - self._Hy_DesnivelIncl - self._E_EspesorZap)
            return self._v_losa + self._v_Incl + self._v_cunha
        else:
            return self._A_Ancho**2 * self._P_Profundidad
            
    def volumen_relleno(self):
        self._v_concr_enterrado = self.volumen_concr()- func.troncopiramidal(self._EP_EspesorPedestal**2, (self._EP_EspesorPedestal + self._Cunha_nvl_terreno)**2, self._AP_AlturaPedestal)
        return self.volumen_excavacion() - self._v_concr_enterrado

    def volumen_suelo_aportante(self):
        self.volumen_relleno()
        return (self._A_Ancho**2 * self._P_Profundidad) - self._v_concr_enterrado

    def volumen_cono(self):
        self._R0_cono = self._P_Profundidad * math.tan(inputOP_['Betha'])
        return func.troncopiramidal(self._A_Ancho**2,
                                   (self._A_Ancho + 2*self._R0_cono)**2,
                                   self._P_Profundidad) - (self._A_Ancho**2 * self._P_Profundidad)

#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#═══#
# CREACIÓN DE OBJETO
_A = 2.4
_P = 3
_AP  = 1
_EP = .5
_E = .4
_Hy = 0.5
# _Hx = None, _Alpha = None

z2 = CimentacionCunha(_AP, _P, _A, _EP, _E, _Hy)

# REACCIONES
FR_perm = FR_ort
FR_event = FR_diag

#     # Equiv
# FR_perm_equiv = func.matriz_equivalente(altura = (z2.g_altura_pedestal + z2.g_prof_zapata), 
#                                     excentricidad = z2.g_excentricidad, 
#                                     num_pata_c = input_z['pata_comp'],
#                                     num_pata_t = input_z['pata_trac'],
#                                     vector = FR_perm)
# FR_event_equiv = func.matriz_equivalente(altura = (z2.g_altura_pedestal + z2.g_prof_zapata), 
#                                     excentricidad = z2.g_excentricidad, 
#                                     num_pata_c = input_z['pata_comp'],
#                                     num_pata_t = input_z['pata_trac'],
#                                     vector = FR_event)

### CHEQUEO

In [27]:
class ChequeoZapataTranselec(): # SE INGRESA EL OBJETO Y OBJETOS FReaction
    def __init__(self, Obj_Zap, reaccion_permanente, reaccion_eventual):
        self._obj_Zap = Obj_Zap #obj
        # CONVERTIR CORTANTE EQUIVALENTE Y VALORES POSITIVOS DE CORTANTE
        self._perm = (func.matriz_equivalente(altura = (self._obj_Zap.g_altura_pedestal + self._obj_Zap.g_prof_zapata), 
                                     excentricidad = self._obj_Zap.g_excentricidad, 
                                     num_pata_c = input_z['pata_comp'],
                                     num_pata_t = input_z['pata_trac'],
                                     vector = reaccion_permanente) )
        self._perm =  func.abs_vectores(self._perm) #F([])
        self._event = (func.matriz_equivalente(altura = (self._obj_Zap.g_altura_pedestal + self._obj_Zap.g_prof_zapata), 
                                     excentricidad = self._obj_Zap.g_excentricidad, 
                                     num_pata_c = input_z['pata_comp'],
                                     num_pata_t = input_z['pata_trac'],
                                     vector = reaccion_eventual) )        
        self._event =  func.abs_vectores(self._event) #F([])
####################
        _peso_concr = np.array([self._obj_Zap.volumen_concr() * - inputOP_['Gamma_concr_arranc'], 0,0]) #[]
        _peso_suelo = np.array([self._obj_Zap.volumen_suelo_aportante() * - input_['Gamma_suelo_arranc'], 0,0]) #[]
        _peso_cono_perm  = func.casos_traccion(self._perm)  * self._obj_Zap.volumen_cono() * -input_['Gamma_suelo_arranc'] #F([])
        _peso_cono_event = func.casos_traccion(self._event) * self._obj_Zap.volumen_cono() * -input_['Gamma_suelo_arranc'] #f([])
        _R_perm  = _peso_concr + _peso_suelo + _peso_cono_perm + self._perm
        _R_event = _peso_concr + _peso_suelo + _peso_cono_event + self._event

        self._R_perm = _R_perm
        self._R_event = _R_event
    
####
# CHEQUEOS PARA TRACCIÓN
    def FS_arranc(self):
        # CONO DE ARRANCAMIENTO
        _peso_concr = np.array([self._obj_Zap.volumen_concr() * -inputOP_['Gamma_concr_arranc'], 0,0]) #[]
        _peso_suelo = np.array([self._obj_Zap.volumen_suelo_aportante() * -input_['Gamma_suelo_arranc'], 0,0]) #[]
        _peso_cono_perm  = func.casos_traccion(self._perm)  * self._obj_Zap.volumen_cono() * -input_['Gamma_suelo_arranc'] #F([])
        _peso_cono_event = func.casos_traccion(self._event) * self._obj_Zap.volumen_cono() * -input_['Gamma_suelo_arranc'] #f([])
        _R_perm  = _peso_concr + _peso_suelo + _peso_cono_perm + self._perm
        _R_event = _peso_concr + _peso_suelo + _peso_cono_event + self._event

        _v, _Rv = self._perm[:,0], _R_perm[:,0]
        _l1 = ['comp' if _v[i] <= 0 else abs(_Rv[i] - _v[i])/_v[i] for i in range(4)]
        _v, _Rv = self._event[:,0], _R_event[:,0]
        _l2 = ['comp' if _v[i] <= 0 else abs(_Rv[i] - _v[i])/_v[i] for i in range(4)]
        _l_cono = [_l1, _l2]
        
        # MEYERHOF ADAMS
        _m = func.m_MeyerhofAdams(input_['Phi'])
        _ku = func.ku_MeyerhofAdams(input_['Phi']) #Jc_Flag SE USA?
        _wconcr = self._obj_Zap.volumen_concr() * inputOP_['Gamma_concr_arranc']
        _wc1 = self._obj_Zap.volumen_suelo_aportante()*input_['Gamma_suelo_arranc']
        _Qu = func.Qu_MeyerhofAdams(Coh = input_['Cohesion'], 
                                    Ancho = self._obj_Zap.g_ancho_zapata, 
                                    Prof = self._obj_Zap.g_prof_zapata, 
                                    eme = _m, 
                                    gamma_s = input_['Gamma_suelo_arranc'], 
                                    phi = input_['Phi'], 
                                    Wcr = _wconcr, 
                                    Wsaport = _wc1)
        _v = self._perm
        _l3 = ['comp' if _v[i].F_vert <= 0 else _Qu/_v[i].F_vert for i in range(4)]
        _v = self._event
        _l4 = ['comp' if _v[i].F_vert <= 0 else _Qu/_v[i].F_vert for i in range(4)]
        _l_MA = [_l3, _l4]
        
        return 1
#     def FS_arranc_Boole(self):
#         return func.min_(self.FS_arranc()) >= param.FS_ARRANC
# #
    def FS_Met_XY_volcam_comp(self):
        1
    def FS_vuelco_Boole(self):
        1
#
#     def FS_deslizamiento(self):
#         _reacciones_fx_fy = self._ort_RT[:, (1,2)]
#         _minima_reaccion_fx_fy = min([j for i in _reacciones_fx_fy for j in i])
#         if _minima_reaccion_fx_fy == 0:
#             return [param.FS_DESLIZAM for i in range(4)]
#         elif _minima_reaccion_fx_fy < 0:
#             return [0 for i in range(4)]
#         else:
#             _FS_matriz = (self._ort_RT + self._ort)[:, (1,2)] / (self._ort)[:, (1,2)]
#             return [min(i) for i in _FS_matriz]
#     def FS_deslizamiento_Boole(self):
#         return func.min_(self.FS_deslizamiento()) >= param.FS_DESLIZAM

Disenho = ChequeoZapataTranselec(z2, FR_perm, FR_event)